In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from surprise import Reader, Dataset, SVD, SVDpp, NMF, SlopeOne, KNNBasic, KNNBaseline, KNNWithMeans, KNNWithZScore, CoClustering, BaselineOnly
from surprise.model_selection.validation import cross_validate

In [ ]:
df = pd.read_csv("data/ReviewRaw.csv")

In [ ]:
df.head()

,customer_id,product_id,name,full_name,created_time,rating,title,content
0,709310,10001012,Lân Nguyễn Hoàng,Lân Nguyễn Hoàng,NaN,3,Ko dùng đc thẻ nhớ,Lúcđầu quên thông tin nên dùng 512gb thì ko đc...
1,10701688,10001012,Nguyễn Khánh Hòa,Nguyễn Khánh Hòa,NaN,5,Cực kì hài lòng,"Tiki giao hàng nhanh. Sản phẩm đúng như mô tả,..."
2,11763074,10001012,Toàn Phạm Khánh,Toàn Phạm Khánh,2019-04-17 15:42:45,5,Cực kì hài lòng,"chất lượng camera rõ nét, chống mưa nắng tuyệt..."
3,9909549,10001012,Nguyen Quang Minh,NaN,NaN,5,Rất hài lòng,"Hàng được đóng gói cẩn thận, giao hàng nhanh ,..."
4,1827148,10001012,Phạm Bá Đức,Phạm Bá Đức,NaN,5,Cực kì hài lòng,"dễ cài đặt, chất lượng tốt, chế độ xem hồng ng..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364099 entries, 0 to 364098
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   customer_id   364099 non-null  int64 
 1   product_id    364099 non-null  int64 
 2   name          363669 non-null  object
 3   full_name     329471 non-null  object
 4   created_time  117340 non-null  object
 5   rating        364099 non-null  int64 
 6   title         364070 non-null  object
 7   content       165794 non-null  object
dtypes: int64(3), object(5)
memory usage: 22.2+ MB


In [ ]:
df.shape

(364099, 8)

In [ ]:
product = pd.read_csv('data/ProductRaw.csv')
product.head()

,item_id,name,description,rating,price,list_price,brand,group,url,image
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,4.0,77000,300000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-inpods-12-cam-bien-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,4.5,132000,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-khong-day-f9-true-w...,https://salt.tikicdn.com/cache/280x280/ts/prod...
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nĐộ p...,4.8,299000,399000,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://chuot-khong-day-logitech-m331-silent-p...,https://salt.tikicdn.com/cache/280x280/media/c...
3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,THÔNG TIN CHI TIẾT\nThương hiệu Acome\nXuất xứ...,4.7,149000,350000,Acome,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://loa-bluetooth-5-0-kiem-dong-ho-bao-thu...,https://salt.tikicdn.com/cache/280x280/ts/prod...
4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,THÔNG TIN CHI TIẾT\nThương hiệu Apple\nXuất xứ...,4.8,5090000,8500000,Apple,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-apple-airpods-pro-t...,https://salt.tikicdn.com/cache/280x280/ts/prod...


In [ ]:
product_select = product[['item_id','name']]
product_select.rename(columns={"name": "product_name"}, inplace=True)

<ipython-input-50-b71a4343c5f5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_select.rename(columns={"name": "product_name"}, inplace=True)


In [ ]:
df = df.merge(product_select, left_on='product_id', right_on='item_id',how='left')

In [ ]:
df.sample(10)

,customer_id,product_id,name,full_name,created_time,rating,title,content,item_id,product_name
318959,12704823,6817797,Nguyễn Đức Khánh,NaN,NaN,4,Hài lòng,Sản phẩm giao đúng như mô tả. Dùng đến nay tốt...,6817797.0,Vỏ hộp đựng ổ cứng SATA 2.0/3.0 2.5 và SSD vỏ ...
92587,5710786,25300127,Nguyễn Phước Lộc,Nguyễn Phước Lộc,2017-09-06 10:47:42,5,Cực kì hài lòng,NaN,25300127.0,Máy Nước Nóng Ariston SMC45PE-VN-LE (4500W)
255604,205725,54637084,Nguyễn Vương Tuấn Kiệt,Nguyễn Vương Tuấn Kiệt,NaN,4,Hài lòng,Mẫu mã đẹp. Thời gian sử dụng pin lâu. Hộp đựn...,54637084.0,Điện thoại Energizer Power Max P20 kiêm sạc dự...
340791,18517774,74240829,Nguyễn Quốc Khánh,Nguyễn Quốc Khánh,2020-09-19 18:38:43,5,Cực kì hài lòng,NaN,74240829.0,Điện thoại Xiaomi POCO M3 - HÀNG CHÍNH HÃNG
271217,8339790,56515332,Phạm Sơn,Phạm Sơn,2018-10-10 19:55:09,5,Cực kì hài lòng,NaN,56515332.0,Dây Cáp Sạc Nhanh USB Type-C Xiaomi 1m Cho And...
21587,6248899,12485554,nguyen trong nhan,nguyen trong nhan,NaN,5,Cực kì hài lòng,Bao da đẹp được gói cẩn thận,12485554.0,Bao da kiêm ốp lưng tự động tắt/mở màn hình th...
49118,15431640,16718181,Nguyn Nam Cao,Nguyn Nam Cao,NaN,5,rất tốt nha mn,"tai nghe chất lượng, tiki giao hàng sớm hơn dự...",16718181.0,Tai nghe Samsung A50 chính hãng
137267,20465783,35723184,Trần Quang Huy,Trần Quang Huy,2021-02-24 22:37:38,2,Không hài lòng,hỏng mất một bên tai mong shop giúp em hoàn trả ạ,35723184.0,Tai nghe Inpod i12 TWS Bluetooth 5.0 cho iPhon...
352132,6411020,7932267,Lê Phạm Quốc Anh,Lê Phạm Quốc Anh,NaN,5,Tốt,"Hàng chuẩn, dùng 1 tháng chưa thấy lỗi lầm gì,...",7932267.0,Camera IP Xiaomi MJSXJ02CM Mi Home Security Ca...
108293,15461991,29466943,Nguyn Huy ng,Nguyn Huy ng,2019-12-14 09:47:47,4,Hài lòng,nghe ko dc hay lắm,29466943.0,Bộ Loa Soundbar 5.1 Bluetooth Karaoke L5 + Loa...


In [ ]:
df_select = df[['product_id','product_name', 'rating', 'customer_id']]

In [ ]:
df_select.sample(10)

,product_id,product_name,rating,customer_id
217899,487524,Thẻ Nhớ Micro SD Samsung Evo Plus 64GB Class 1...,5,255116
35516,14868418,Tai Nghe Bluetooth Không Dây QCY-T1 Mini Hàng ...,5,13735030
150273,379462,Loa Vi Tính Microlab B51 2.0 (4W) - Hàng Chính...,4,263848
175195,42644889,Loa Bluetooth GUTEK C3 MINI Nghe Nhạc Cầm Tay ...,3,10361780
194964,465219,Thiết Bị Phát Wifi Di Động 4G LTE-Advanced TP-...,3,5862766
253932,54478250,Camera Wifi thông minh EZVIZ C6N 1080P (CS-C6N...,4,10833569
137186,35723184,Tai nghe Inpod i12 TWS Bluetooth 5.0 cho iPhon...,2,12473614
186839,45100535,[[ KÈM VIDEO - Chỉ 30p sạc 50% pin iPhone X ]]...,4,492893
26882,13572054,Bộ phụ kiện 5 món gồm 1 bao silicon bảo vệ tai...,5,16153039
353329,8021522,Tay cầm chống rung cho điện thoại Gimbal Zhiyu...,5,6710309


In [ ]:
df.duplicated().sum()

13534

In [ ]:
df_select.drop_duplicates(inplace=True)

<ipython-input-67-83f0ff8a3b13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select.drop_duplicates(inplace=True)


In [ ]:
df_select.isnull().sum()

product_id        0
product_name    660
rating            0
customer_id       0
dtype: int64

In [ ]:
df_select.dropna(inplace=True)

<ipython-input-69-9288ecfd02fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select.dropna(inplace=True)


In [ ]:
df_select.shape

(359551, 4)

In [ ]:
n_ratings = len(df_select)
n_products = len(df_select['product_id'].unique())
n_users = len(df_select['customer_id'].unique())

In [ ]:
display(n_ratings, n_products, n_users)

359551

4214

251149

In [ ]:
reader = Reader()
data = Dataset.load_from_df(df_select[['customer_id', 'product_id', 'rating']], reader)

In [ ]:
svd = SVD()
svd_pp = SVDpp()
nmf = NMF()
bo = BaselineOnly()
so = SlopeOne()
cc = CoClustering()

In [ ]:
alg_lst = [svd, svd_pp, nmf, bo, so,cc]

In [ ]:
for i in alg_lst:
  # Run 5-fold cross-validation and print results
  results = cross_validate(i, data, measures=['RMSE', 'MAE'],cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9809  0.9727  0.9724  0.9748  0.9752  0.9752  0.0030  
MAE (testset)     0.6881  0.6844  0.6846  0.6860  0.6854  0.6857  0.0013  
Fit time          8.72    8.26    7.40    8.09    8.81    8.26    0.51    
Test time         0.45    0.76    0.47    0.46    1.12    0.65    0.26    
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9845  0.9845  0.9946  0.9897  0.9825  0.9871  0.0044  
MAE (testset)     0.6619  0.6587  0.6660  0.6618  0.6592  0.6615  0.0026  
Fit time          6.26    6.45    8.00    8.02    6.60    7.07    0.78    
Test time         2.26    0.87    0.85    0.89    0.91    1.15    0.55    
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (tests

Based on the average RMSE, we choose the BaselineOnly algorithm with RMSE ~ 0.97, this algorithm has the lowest RMSE of all algorithms.

In [ ]:
# If the results are OK => getting full dataset => fit model
trainset = data.build_full_trainset()
start_time = datetime.now()
bo.fit(trainset)
end_time = datetime.now()
print('Time taken', end_time - start_time)

Estimating biases using als...
Time taken 0:00:02.333593


Thời gian training model khá nhanh, chỉ mất khoảng 2 giây

### Recommender for a specific user

In [ ]:
df_score = df_select[["product_id","product_name"]]

In [ ]:
df_score

,product_id,product_name
0,10001012,Camera IP Wifi Ngoài Trời Ezviz CS-CV310 720P ...
1,10001012,Camera IP Wifi Ngoài Trời Ezviz CS-CV310 720P ...
2,10001012,Camera IP Wifi Ngoài Trời Ezviz CS-CV310 720P ...
3,10001012,Camera IP Wifi Ngoài Trời Ezviz CS-CV310 720P ...
4,10001012,Camera IP Wifi Ngoài Trời Ezviz CS-CV310 720P ...
...,...,...
375279,9996258,Kẹp ba lô gắn GoPro Hero xoay 360 độ
375280,9996258,Kẹp ba lô gắn GoPro Hero xoay 360 độ
375281,9996258,Kẹp ba lô gắn GoPro Hero xoay 360 độ
375282,9996258,Kẹp ba lô gắn GoPro Hero xoay 360 độ


- Once the model has been evaluated to our satisfaction, then we can re-train the model using the entire training dataset

In [ ]:
# The following are the top 5 movies to be recommended to the user with userId
# To recommend products (i.e., movies) to the given user,
# we can sort the list of movies in decreasing order of predicted ratings
# and take the top N movies as recommendations:
def recommendation_for_user(df_score,customer_id,number_products):
  df_score['EstimateScore'] = df_score['product_id'].apply(lambda x: bo.predict(customer_id, x).est) # est: get EstimateScore
  df_score = df_score.sort_values(by=['EstimateScore'], ascending=False)
  df_score = df_score.drop_duplicates()
  return df_score.head(number_products)

In [ ]:
# Suggestion for customer_id = 50
start_time = datetime.now()
result_50 = recommendation_for_user(df_score,50,5)
end_time = datetime.now()
print('Time taken',end_time - start_time)

Time taken 0:00:01.215046


<ipython-input-107-3dd37406baf2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score['EstimateScore'] = df_score['product_id'].apply(lambda x: bo.predict(customer_id, x).est) # est: get EstimateScore


In [ ]:
result_50

,product_id,product_name,EstimateScore
245476,53080935,Phần mềm Microsoft 365 Personal English APAC E...,4.885969
317151,68025746,Đồng Hồ Thông Minh Apple Watch SE GPS Only Alu...,4.885538
337481,73179180,Thẻ Nhớ microSD SanDisk Ultra 128GB UHS-I - 10...,4.878008
326122,70771651,Điện Thoại iPhone 12 Pro Max 128GB - Hàng Chín...,4.870738
332815,71896003,Đồng Hồ Thông Minh Apple Watch Series 6 LTE GP...,4.869409


In [ ]:
# Some products were bought by customer_id=50
df[df.customer_id==50]

,customer_id,product_id,name,full_name,created_time,rating,title,content,item_id,product_name
132319,50,35580965,Huỳnh Hải Huỳnh,Huỳnh Hải Huỳnh,NaN,4,Hài lòng,"Good, bass hơi rè nếu volume to",35580965.0,Loa thanh soundbar LG 2.1 SL4 300W - Hàng chín...
370475,50,964243,Huỳnh Hải Huỳnh,NaN,NaN,5,Giao hàng nhanh,"Giao hàng nhanh. Đóng gói đẹp. Hài lòng, Giao ...",964243.0,Ổ Cứng SSD WD Green 240GB 3D NAND - WDS240G2G0...


In [ ]:
# Suggestion for customer_id = 1499390
start_time = datetime.now()
result_1499390 = recommendation_for_user(df_score,1499390,5)
end_time = datetime.now()
print('Time taken',end_time - start_time)

Time taken 0:00:03.121464


<ipython-input-107-3dd37406baf2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score['EstimateScore'] = df_score['product_id'].apply(lambda x: bo.predict(customer_id, x).est) # est: get EstimateScore


In [ ]:
result_1499390

,product_id,product_name,EstimateScore
245476,53080935,Phần mềm Microsoft 365 Personal English APAC E...,4.953082
317151,68025746,Đồng Hồ Thông Minh Apple Watch SE GPS Only Alu...,4.952651
337481,73179180,Thẻ Nhớ microSD SanDisk Ultra 128GB UHS-I - 10...,4.945121
326122,70771651,Điện Thoại iPhone 12 Pro Max 128GB - Hàng Chín...,4.937851
332815,71896003,Đồng Hồ Thông Minh Apple Watch Series 6 LTE GP...,4.936522


In [ ]:
# Some products were bought by customer_id=1499390
df[df.customer_id==1499390]

,customer_id,product_id,name,full_name,created_time,rating,title,content,item_id,product_name
46871,1499390,1610105,Lý Trần,Lý Trần,NaN,5,Cực kì hài lòng,NaN,1610105.0,Ổ Cứng SSD Sata III 2.5 inch 500GB Samsung 860...
132355,1499390,35580965,Lý Trần,Lý Trần,2016-07-11 12:49:26,5,Cực kì hài lòng,NaN,35580965.0,Loa thanh soundbar LG 2.1 SL4 300W - Hàng chín...
153231,1499390,3831591,Lý Trần,Lý Trần,NaN,4,Hài lòng,NaN,3831591.0,Ổ Cứng HDD Box ORICO USB3.0/2.5 - 2577U3 - Hàn...
189282,1499390,455445,Lý Trần,Lý Trần,NaN,5,Cực kì hài lòng,NaN,455445.0,Bộ Kích Sóng Wifi Repeater 300Mbps TP-Link TL-...
370770,1499390,964243,Lý Trần,Lý Trần,NaN,5,Cực kì hài lòng,NaN,964243.0,Ổ Cứng SSD WD Green 240GB 3D NAND - WDS240G2G0...


### Conclusion
- Using Surprise with the BaselineOnly algorithm, the model run time is quite fast, only taking about 2 seconds and the average RMSE ~ 0.97 => The model is quite good
- the recommended products for users lack suitability, and the recommendations tend to be similar for different users, similar to the ALS algorithm.
#### => We will conduct a thorough reevaluation of the dataset and explore alternative recommendation models.
